# Build AI agents with Writer models

Writer models support advanced tool-calling capabilities, enabling complex multi-step workflows and agentic actions. This notebook shows you how to use Writer models and LlamaIndex to build an AI agent on AWS. 

## Install and import the required libraries

In [ ]:
!pip install --upgrade pip
!pip install -U llama-index
!pip install llama-index-llms-bedrock-converse
!pip install llama_index-tools-arxiv

In [13]:
from llama_index.core import PromptTemplate, Settings
from llama_index.core.agent import ReActAgent
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.memory.types import ChatMessage
from llama_index.core.tools import FunctionTool

from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.tools.arxiv.base import ArxivToolSpec

## Define functions as tools

In [14]:
def github_search(topic: str, num_results: int = 3, sort_by: str = "stars") -> list:
    """Search GitHub repositories by topic"""
    url = f"https://api.github.com/search/repositories?q=topic:{topic}&sort={sort_by}&order=desc"
    response = requests.get(url).json()
    code_repos = [
        {
            'html_url': item['html_url'],
            'description': item['description'],
            'stargazers_count': item['stargazers_count'],
        }
        for item in response['items'][:num_results]
    ]
    return code_repos

def news_search(topic: str, num_results: int = 3) -> list:
    """Search for news using NewsAPI"""
    API_KEY = 'f2a5f3027b2c46bd9d313c7c39b8ea06'
    BASE_URL = 'https://newsapi.org/v2/everything'
    
    try:
        params = {
            'q': topic,
            'apiKey': API_KEY,
            'sortBy': 'publishedAt',
            'language': 'en',
            'domains': 'techcrunch.com,wired.com,theverge.com,venturebeat.com,arstechnica.com,deeplearning.ai',
            'pageSize': num_results
        }
        
        response = requests.get(BASE_URL, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if data['status'] == 'ok' and data['totalResults'] > 0:
                articles = []
                for article in data['articles'][:num_results]:
                    articles.append({
                        'title': article['title'],
                        'link': article['url'],
                        'published': article['publishedAt'],
                        'summary': article['description'],
                        'content': article['content']
                    })
                return articles
            else:
                print(f"No results found for topic: {topic}")
                return []
        elif response.status_code == 429:
            print("Rate limit exceeded")
            return []
        else:
            print(f"Error: {response.status_code}")
            return []
    except Exception as e:
        print(f"Error fetching news: {str(e)}")
        return []

## Create tools

In [15]:
github_tool = FunctionTool.from_defaults(fn=github_search)
news_tool = FunctionTool.from_defaults(fn=news_search)
arxiv_tool = ArxivToolSpec() # Connect to ArXiv and search for recent papers 

api_tools = arxiv_tool.to_tool_list()
api_tools.extend([news_tool, github_tool])

## Select the Writer model as LLM for the AI agent

In [16]:
model_id = "us.writer.palmyra-x5-v1:0"
# model_id = "arn:aws:bedrock:us-west-2:YOUR AWS ACCOUNT ID:inference-profile/us.writer.palmyra-x5-v1:0"

llm = BedrockConverse(model=model_id, max_tokens=4000)
Settings.llm = BedrockConverse(model=model_id, max_tokens=4000)

# Create memory buffer
memory = ChatMemoryBuffer.from_defaults(token_limit=4096)

## Create the AI agent

In [18]:
# System prompt 
system_prompt = """
You are a friendly and engaging technology expert with access to the GitHub API, arXiv API, and TechCrunch API. 
Think of yourself as an enthusiastic tech researcher having a conversation with a colleague.
Your news_search tool returns the absolute latest articles, often just minutes or hours old.
## Tools
You have access to these tools and are responsible for using them in any sequence you deem appropriate:
{tool_desc}

## Output Format
To answer the question, please use the following format:

Thought: Explain your thinking process conversationally
Action: tool name (one of {tool_names})
Action Input: the input to the tool in JSON format

If a tool returns no results, try:
1. Using a different tool
2. Modifying your search terms
3. If all tools fail, acknowledge that you couldn't find the information

When you get an empty result, don't repeat the same exact search - try a different approach.

Continue until you can answer or determine you can't find the information, then use:

Thought: [Explain your conclusion]
Answer: [Provide your findings or explain why you couldn't find the information]

## Additional Rules
- If a search returns no results, try alternative search terms or different tools
- Don't repeat the same search multiple times
- If no information is found after trying different approaches, acknowledge the limitation
- Use a conversational, engaging tone
- Connect ideas across different sources when relevant
- Highlight what's exciting or surprising
- Keep it concise but interesting
- Always check the dates of the articles and mention if they're not recent
- If articles are more than 30 days old, try another search or tool
- Format dates in a user-friendly way (e.g., "2 days ago" or "March 15, 2024")
- Keep responses clear and concise, but when the answer should be longer format feel free to make it so if it helps the user understand the full answer.

## Current Conversation
Below is the current conversation consisting of interleaving human and assistant messages.
"""

# Create and configure the agent
react_system_prompt = PromptTemplate(template=system_prompt)
agent = ReActAgent.from_tools(
    api_tools,
    llm=llm,
    verbose=True,
    max_iterations=10,
    memory=memory
)
agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})
agent.reset()

In [21]:
response = agent.chat("find a paper on Model Context Protocol and summarize the paper")

> Running step b48500af-0945-4499-8b4b-02e6f8b8e423. Step input: find a paper on Model Context Protocol and summarize the paper
Thought: To find a paper on Model Context Protocol, I'll start by querying arXiv.
Action: arxiv_query
Action Input: {'query': 'Model Context Protocol', 'sort_by': 'relevance'}
Observation: [Document(id_='d5bbcb73-7541-4c06-a54d-1d359f907003', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='http://arxiv.org/pdf/2505.02279v2: A survey of agent interoperability protocols: Model Context Protocol (MCP), Agent Communication Protocol (ACP), Agent-to-Agent Protocol (A2A), and Agent Network Protocol (ANP)\nLarge language model powered autonomous agents demand robust, standardized\nprotocols to integrate tools, share contextual data, and coordinate tasks\nacross heterogeneous systems. Ad

In [23]:
response = agent.chat("find github sample codes for writer's palmyra-x5 model")

> Running step 1cca2c4b-3e3b-4e73-a33f-b12dbd44b6ab. Step input: find github sample codes for writer's palmyra-x5 model
Thought: To find GitHub sample codes for the Writer's Palmyra-X5 model, I'll start by searching for the model on GitHub.
Action: github_search
Action Input: {'topic': 'Palmyra-X5 Writer', 'num_results': 3}
Observation: Error: name 'requests' is not defined
> Running step 9b788830-ad5e-4c2f-ad8a-ac42a76762b8. Step input: None
Thought: It seems like there's an issue with the GitHub search tool. Let's try to search for the Palmyra-X5 model on arXiv to see if there are any related papers or references to GitHub repositories.
Action: arxiv_query
Action Input: {'query': 'Palmyra-X5 Writer model', 'sort_by': 'relevance'}
Observation: [Document(id_='5cc285da-8d9e-432a-ac0d-6eed4247a3b1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=Media